In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import plotly.express as px
from AnimalShelter import AnimalShelter
import dash

# Dashboard Initialization
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Dashboard Layout
app.layout = html.Div([
    html.Center(html.B(html.H1("CS-340 Dashboard - ADeMarco"))),

    html.Center(
        html.Img(
            src="data:image/png;base64,{}".format(base64.b64encode(open("GSL.png", "rb").read()).decode()),
            style={"height": "200px", "display": "block", "margin": "auto"}
        )
    ),

    html.Hr(),

    # User Input for Authentication
    html.Div([
        html.Label("Username:"),
        dcc.Input(id="input_user", type="text", placeholder="Enter Username"),

        html.Label("Password:"),
        dcc.Input(id="input_passwd", type="password", placeholder="Enter Password"),
        
        html.Button("Login", id="login-button", n_clicks=0),
    ]),

    html.Hr(),

    # Placeholder for Data Table
    html.Div(id="data-table-container"),

    html.Hr(),

    # Interactive Filters
    html.Div(id="filter-section", style={"display": "none"}, children=[
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id="rescue-type",
            options=[
                {"label": "Water Rescue", "value": "Water"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster"},
                {"label": "Reset Filters", "value": "Reset"}
            ],
            value="Reset"
        ),

        html.Label("Preferred Dog Breeds:"),
        dcc.Dropdown(
            id="breed-filter",
            multi=True,
            value=[]
        ),

        html.Label("Preferred Sex:"),
        dcc.RadioItems(
            id="sex-filter",
            options=[
                {"label": "Intact Male", "value": "Intact Male"},
                {"label": "Intact Female", "value": "Intact Female"}
            ],
            value="Intact Male"
        ),

        html.Label("Select Training Age Range (Weeks):"),
        dcc.RangeSlider(
            id="age-filter",
            min=20, max=300, step=1,
            marks={20: "20", 50: "50", 100: "100", 150: "150", 200: "200", 250: "250", 300: "300"},
            value=[20, 300]
        ),

        html.Br(),

        # Apply Filter Button Below Filters
        html.Button("Apply Filter", id="apply-filter-button", n_clicks=0, style={"margin-top": "10px"})
    ]),

    html.Br(),
    html.Hr(),

    # Charts Section
    html.Div(className="row", style={"display": "flex"}, children=[
        html.Div(id="graph-id", className="col s12 m6"),  # Pie Chart
        html.Div(id="map-id", className="col s12 m6")  # Map
    ])
])

# Unified Callback for Login and Filtering
@app.callback(
    [Output("data-table-container", "children"),
     Output("filter-section", "style"),
     Output("graph-id", "children"),
     Output("map-id", "children")],  # ✅ Ensure map updates
    [Input("login-button", "n_clicks"),
     Input("apply-filter-button", "n_clicks")],
    [State("input_user", "value"),
     State("input_passwd", "value"),
     State("breed-filter", "value"),
     State("sex-filter", "value"),
     State("age-filter", "value")]
)
def authenticate_and_filter_data(login_clicks, filter_clicks, inputUser, inputPass, breed_filter, sex_filter, age_filter):
    """Handles user authentication, filtering, and updates the pie chart and map."""

    ctx = dash.callback_context
    if not ctx.triggered:
        return html.Div("Please enter credentials and click Login."), {"display": "none"}, "No data available.", "No data available."

    button_clicked = ctx.triggered[0]["prop_id"].split(".")[0]

    try:
        print("Button clicked:", button_clicked)

        # Authenticate user
        db = AnimalShelter(inputUser, inputPass)

        # Construct MongoDB query
        query = {}

        # If Apply Filter was clicked, apply filters
        if button_clicked == "apply-filter-button":
            if breed_filter:
                query["breed"] = {"$in": breed_filter}

            if sex_filter:
                query["sex_upon_outcome"] = sex_filter

            if age_filter:
                query["age_upon_outcome_in_weeks"] = {"$gte": age_filter[0], "$lte": age_filter[1]}

        print("MongoDB Query:", query)

        # Retrieve data
        df = pd.DataFrame.from_records(db.read(query))
        print("Records Retrieved:", len(df))

        if df.empty:
            return html.Div("No data found."), {"display": "block"}, "No data available.", "No data available."

        if "_id" in df.columns:
            df.drop(columns=["_id"], inplace=True)

        # ✅ Update pie chart
        pie_chart = dcc.Graph(
            figure=px.pie(df, names="breed", title="Breed Distribution")
        )

        # ✅ Update map
        map_view = []
        if not df.empty:
            row = 0  # Default to first row
            map_view = [
                dl.Map(style={"width": "1000px", "height": "500px"}, center=[30.75,-97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(position=[df.iloc[row]["location_lat"], df.iloc[row]["location_long"]],
                        children=[
                            dl.Tooltip(df.iloc[row]["breed"]),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(df.iloc[row]["name"])
                            ])
                        ])
                ])
            ]

        return (
            dash_table.DataTable(
                id="datatable-id",
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict("records"),
                page_size=10,
                sort_action="native",
                filter_action="native",
                row_selectable="single",
                selected_rows=[0]
            ),
            {"display": "block"},
            pie_chart,
            map_view
        )

    except Exception as e:
        print("Error:", e)
        return html.Div("Error: " + str(e), style={"color": "red"}), {"display": "none"}, "No data available.", "No data available."

# Callback to Update Breed List
@app.callback(
    Output("breed-filter", "options"),
    Input("rescue-type", "value")
)
def update_breed_options(rescue_type):
    """Ensures breed dropdown updates when a rescue type is selected."""

    breed_options = {
        "Water": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "Mountain": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "Disaster": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
    }

    return [{"label": breed, "value": breed} for breed in breed_options.get(rescue_type, [])]

# Run App
app.run_server(debug=True)


Dash app running on http://127.0.0.1:13652/
Button clicked: login-button
MongoDB Query: {}
MongoDB returned 10002 documents.
Records Retrieved: 10002
Button clicked: apply-filter-button
MongoDB Query: {'breed': {'$in': ['Labrador Retriever Mix']}, 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
MongoDB returned 22 documents.
Records Retrieved: 22
Button clicked: apply-filter-button
MongoDB Query: {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever']}, 'sex_upon_outcome': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 50}}
MongoDB returned 12 documents.
Records Retrieved: 12
